In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import time
import glob
import scipy
import numba
import torch
import tqdm
import numpy as np
import pandas as pd

import seaborn as sns

import scipy.io as sio
from scipy.stats import moment
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree
from toolz.curried import pipe, curry

In [3]:
import ase.io as aio
from ase import Atoms
from ase.neighborlist import neighbor_list

from asap3.analysis import PTM
from asap3.analysis.localstructure import RestrictedCNA, GuessLatticeConstant

from sklearn.decomposition import PCA

In [6]:
import utilities as utils

ModuleNotFoundError: No module named 'utilities'

In [ ]:
import matplotlib as mpl
mpl.rcParams['lines.linewidth'] = 2
mpl.rcParams['lines.color'] = 'r'
plt.style.use('ggplot')
mpl.rcParams['figure.figsize'] = [6.0, 4.50]
mpl.rcParams['figure.dpi'] = 80
mpl.rcParams['savefig.dpi'] = 100

mpl.rcParams['font.size'] = 14
mpl.rcParams['legend.fontsize'] = 'large'
mpl.rcParams['figure.titlesize'] = 'medium'

In [6]:
folder = "structs"
if not os.path.exists(folder):
    os.mkdir(folder)
    
if not os.path.exists("rmsds"):
    os.mkdir("rmsds")

In [7]:
runs = np.loadtxt("runs.txt").astype(int)
runs.shape

(77,)

## Load lammps dump file

In [8]:
%%time
run = runs[1]
f_dump = "../new-data/CK_Cu_ACC_DUMP/dump.arc.{}".format(run)
atoms = aio.read(f_dump, format="lammps-dump-text")
atoms.set_pbc([1, 1, 1])
f_struct = "{}/sdump.arc.{}_atom.pkl".format(folder, run)
utils.save_file(f_struct, obj=atoms)

CPU times: user 32.3 s, sys: 1.16 s, total: 33.5 s
Wall time: 33.6 s


## Utilize PTM to identify 
[link to PTM documentation](https://wiki.fysik.dtu.dk/asap/Local%20crystalline%20order)

In [17]:
%%time
ptm = PTM(atoms, rmsd_max=0.10, cutoff=8)
tags = ptm['structure']
atoms.set_tags(tags)


idx = np.where(tags != 1)[0]
print(len(idx))
coords_irr = atoms.get_positions()[idx]

rmsds = ptm['rmsd']

f_rmsd = "{}/dump.arc.{}_rmsd.pkl".format("rmsds", run)
utils.save_file(f_rmsd, obj=rmsds)

In [16]:
sns.distplot(rmsds, kde=False, bins=200)
plt.xlim([0.0, 0.20])
plt.grid(False)
plt.show()

In [ ]:
sns.distplot(rmsds, kde=False, bins=100)
plt.xlim([0.0, 0.10])
plt.grid(False)
plt.show()

In [18]:
def save_struct_data(run):
    
    f_dump = "../new-data/CK_Cu_ACC_DUMP/dump.arc.{}".format(run)
    print(f_dump)
    
    atoms = aio.read(f_dump, format="lammps-dump-text")
    atoms.set_pbc([1, 1, 1])
    ptm = PTM(atoms, rmsd_max=0.10, cutoff=8)
    atoms.set_tags(ptm['structure'])
    
    f_struct = "{}/dump.arc.{}_atom.pkl".format(folder, run)
    utils.save_file(f_struct, obj=atoms)
    
    rmsds = ptm['rmsd']

    f_rmsd = "{}/dump.arc.{}_rmsd.pkl".format("rmsds", run)
    utils.save_file(f_rmsd, obj=rmsds)

In [61]:
%%time
# save_struct_data(runs[0])

CPU times: user 2min 8s, sys: 2.05 s, total: 2min 10s
Wall time: 2min 10s


In [ ]:
import multiprocessing as mp
n_proc = 8
with mp.Pool(n_proc) as p:
    print(p.map(save_struct_data, runs))

../new-data/CK_Cu_ACC_DUMP/dump.arc.3000
../new-data/CK_Cu_ACC_DUMP/dump.arc.0
../new-data/CK_Cu_ACC_DUMP/dump.arc.1500
../new-data/CK_Cu_ACC_DUMP/dump.arc.4500
../new-data/CK_Cu_ACC_DUMP/dump.arc.7500
../new-data/CK_Cu_ACC_DUMP/dump.arc.6000
../new-data/CK_Cu_ACC_DUMP/dump.arc.9000
../new-data/CK_Cu_ACC_DUMP/dump.arc.10500
../new-data/CK_Cu_ACC_DUMP/dump.arc.500
../new-data/CK_Cu_ACC_DUMP/dump.arc.2000
../new-data/CK_Cu_ACC_DUMP/dump.arc.3500
../new-data/CK_Cu_ACC_DUMP/dump.arc.5000
../new-data/CK_Cu_ACC_DUMP/dump.arc.6500
../new-data/CK_Cu_ACC_DUMP/dump.arc.8000
../new-data/CK_Cu_ACC_DUMP/dump.arc.9500
../new-data/CK_Cu_ACC_DUMP/dump.arc.11000
../new-data/CK_Cu_ACC_DUMP/dump.arc.1000
../new-data/CK_Cu_ACC_DUMP/dump.arc.2500
../new-data/CK_Cu_ACC_DUMP/dump.arc.4000
../new-data/CK_Cu_ACC_DUMP/dump.arc.5500
../new-data/CK_Cu_ACC_DUMP/dump.arc.7000
../new-data/CK_Cu_ACC_DUMP/dump.arc.8500
../new-data/CK_Cu_ACC_DUMP/dump.arc.10000
../new-data/CK_Cu_ACC_DUMP/dump.arc.11500
../new-data/CK_C